In [1]:
import pandas as pd
import numpy as np
from sklearn import metrics
from sklearn.utils import resample
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import classification_report
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC
from sklearn.decomposition import PCA

In [2]:
new_train=pd.read_csv('/home/vboxuser/Downloads/train_split_new.csv')
new_test=pd.read_csv('/home/vboxuser/Downloads/test_split_new.csv')

In [3]:
new_train.head()

,Unnamed: 0,Participant_ID,Gender,Class,PHQ_Score,PCL-C (PTSD),PTSD Severity
0,0,302,0,0,4,0,28
1,1,303,1,0,0,0,17
2,2,304,1,0,6,0,20
3,3,305,0,0,7,0,28
4,4,307,1,0,4,0,23


In [4]:
new_train.rename(columns={'Participant_ID':'speaker'},inplace=True)

In [5]:
#we upsample the minority class to majority class 
df_1=new_train[new_train['Class']==1]
df_other=new_train[new_train['Class']!=1]
df_upsampled_new=resample(df_1,random_state=42,n_samples=113,replace=True)
df_train_upsampled_new=pd.concat([df_upsampled_new,df_other])
df_train_upsampled_new.reset_index(inplace=True)

In [6]:
new_train['Class'].value_counts()

0    113
1     50
Name: Class, dtype: int64

In [7]:
#statistical analysis on features:
df=pd.read_csv('/home/vboxuser/VoiceFiles/ImageFolder/ImageFolderopensmile_daic_dev_stat.csv')
#train and test data:
df_train=pd.read_csv('/home/vboxuser/VoiceFiles/opensmile_daic_train.csv')
df_test=pd.read_csv('/home/vboxuser/VoiceFiles/opensmile_daic_test.csv')

In [8]:
#from the {ImageFolder..} file obtained earlier from feature_analysis we take those values that have p_val<0.05 and effect_size>|0.4|
req_features=df[(df['p_HC_DP']<0.05) & (abs(df['p_HC_DP.1'])>0.4)]

In [9]:
feature_list=req_features['Unnamed: 0'].tolist()
print(feature_list)

['mfcc4_sma3_amean', 'F1amplitudeLogRelF0_sma3nz_amean', 'F1amplitudeLogRelF0_sma3nz_stddevNorm', 'F2amplitudeLogRelF0_sma3nz_amean', 'F3amplitudeLogRelF0_sma3nz_amean', 'alphaRatioV_sma3nz_amean', 'alphaRatioV_sma3nz_stddevNorm', 'hammarbergIndexV_sma3nz_amean', 'slopeUV0-500_sma3nz_amean', 'MeanUnvoicedSegmentLength', 'StddevUnvoicedSegmentLength']


In [10]:
#replace string values to 0s & 1s in target column
df_train.replace({'class':{"HC":0,"DP":1}},inplace=True)

In [11]:
df_train.head(2)

,file,start,end,F0semitoneFrom27.5Hz_sma3nz_amean,F0semitoneFrom27.5Hz_sma3nz_stddevNorm,F0semitoneFrom27.5Hz_sma3nz_percentile20.0,F0semitoneFrom27.5Hz_sma3nz_percentile50.0,F0semitoneFrom27.5Hz_sma3nz_percentile80.0,F0semitoneFrom27.5Hz_sma3nz_pctlrange0-2,F0semitoneFrom27.5Hz_sma3nz_meanRisingSlope,...,slopeUV500-1500_sma3nz_amean,spectralFluxUV_sma3nz_amean,loudnessPeaksPerSec,VoicedSegmentsPerSec,MeanVoicedSegmentLengthSec,StddevVoicedSegmentLengthSec,MeanUnvoicedSegmentLength,StddevUnvoicedSegmentLength,equivalentSoundLevel_dBp,class
0,/home/biswajit/Documents/jupyter_scribe/AVEC_2...,0 days,0 days 00:03:40.840000,23.210290,0.143984,21.653550,23.158566,24.440777,2.787228,234.11403,...,-0.007202,0.030223,2.391867,2.183882,0.271206,0.232511,0.161118,0.160292,-43.877804,0
1,/home/biswajit/Documents/jupyter_scribe/AVEC_2...,0 days,0 days 00:10:18.940000,31.877468,0.167992,30.627243,32.708360,34.913250,4.286007,346.58624,...,-0.012183,0.037185,3.042817,1.619538,0.269370,0.239770,0.142960,0.153230,-37.614265,0


In [12]:
for i in range (0,len(df_train)):
    df_train['file'][i]=df_train['file'][i][80:83]

/tmp/ipykernel_5720/936947119.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_train['file'][i]=df_train['file'][i][80:83]


In [13]:
df_train.rename(columns={'file':'speaker'},inplace=True)

In [14]:
df_train['speaker']=df_train['speaker'].astype(int)
new_train['speaker']=new_train['speaker'].astype(int)

In [16]:
df=pd.merge(df_train,new_train,on='speaker',how='inner')
df.head(3)

,speaker,start,end,F0semitoneFrom27.5Hz_sma3nz_amean,F0semitoneFrom27.5Hz_sma3nz_stddevNorm,F0semitoneFrom27.5Hz_sma3nz_percentile20.0,F0semitoneFrom27.5Hz_sma3nz_percentile50.0,F0semitoneFrom27.5Hz_sma3nz_percentile80.0,F0semitoneFrom27.5Hz_sma3nz_pctlrange0-2,F0semitoneFrom27.5Hz_sma3nz_meanRisingSlope,...,MeanUnvoicedSegmentLength,StddevUnvoicedSegmentLength,equivalentSoundLevel_dBp,class,Unnamed: 0,Gender,Class,PHQ_Score,PCL-C (PTSD),PTSD Severity
0,302,0 days,0 days 00:03:40.840000,23.210290,0.143984,21.653550,23.158566,24.440777,2.787228,234.11403,...,0.161118,0.160292,-43.877804,0,0,0,0,4,0,28
1,303,0 days,0 days 00:10:18.940000,31.877468,0.167992,30.627243,32.708360,34.913250,4.286007,346.58624,...,0.142960,0.153230,-37.614265,0,1,1,0,0,0,17
2,304,0 days,0 days 00:05:53.660000,33.147354,0.180221,31.954395,34.013270,36.606310,4.651916,401.54807,...,0.157567,0.167011,-39.449467,0,2,1,0,6,0,20


In [17]:
df.columns

Index(['speaker', 'start', 'end', 'F0semitoneFrom27.5Hz_sma3nz_amean',
       'F0semitoneFrom27.5Hz_sma3nz_stddevNorm',
       'F0semitoneFrom27.5Hz_sma3nz_percentile20.0',
       'F0semitoneFrom27.5Hz_sma3nz_percentile50.0',
       'F0semitoneFrom27.5Hz_sma3nz_percentile80.0',
       'F0semitoneFrom27.5Hz_sma3nz_pctlrange0-2',
       'F0semitoneFrom27.5Hz_sma3nz_meanRisingSlope',
       'F0semitoneFrom27.5Hz_sma3nz_stddevRisingSlope',
       'F0semitoneFrom27.5Hz_sma3nz_meanFallingSlope',
       'F0semitoneFrom27.5Hz_sma3nz_stddevFallingSlope', 'loudness_sma3_amean',
       'loudness_sma3_stddevNorm', 'loudness_sma3_percentile20.0',
       'loudness_sma3_percentile50.0', 'loudness_sma3_percentile80.0',
       'loudness_sma3_pctlrange0-2', 'loudness_sma3_meanRisingSlope',
       'loudness_sma3_stddevRisingSlope', 'loudness_sma3_meanFallingSlope',
       'loudness_sma3_stddevFallingSlope', 'spectralFlux_sma3_amean',
       'spectralFlux_sma3_stddevNorm', 'mfcc1_sma3_amean',
       'mfcc

In [18]:
df.drop(['Class'],axis=1,inplace=True)

In [23]:
df_train_upsampled['class'].value_counts()

1    126
0    126
Name: class, dtype: int64

In [19]:
#we upsample the minority class to majority class 
df_1=df[df['class']==1]
df_other=df[df['class']!=1]
df_upsampled=resample(df_1,random_state=42,n_samples=126,replace=True)
df_train_upsampled=pd.concat([df_upsampled,df_other])
df_train_upsampled.reset_index(inplace=True)

In [21]:
df_train_upsampled['class'].value_counts()

1    126
0    126
Name: class, dtype: int64

In [24]:
X_train_resampled=df_train_upsampled[df_train_upsampled.columns.intersection(feature_list)]
Y_train_resampled=df_train_upsampled['class']
sp=df_train_upsampled['speaker']

In [26]:
#for normalization:
std_Scaler=StandardScaler()
std_Scaler

StandardScaler()

In [27]:
#standard scalar normalization on feature set:
X=std_Scaler.fit_transform(X_train_resampled)
Y=Y_train_resampled

In [29]:
#checking if normalized data has a mean~0 & standard deviation~1:
print(np.mean(X))
print(np.std(X))

-8.394759955318644e-17
1.0


In [30]:
#obtaining X & Y for test data:
X_test=df_test[df_test.columns.intersection(feature_list)]
df_test.replace({'class':{"HC":0,"DP":1}},inplace=True)
Y_test=df_test['class']

In [31]:
pca=PCA(n_components=4)

x_trainpca=pca.fit_transform(X)
x_testpca=pca.transform(X_test)

##convert principal components for each sample from numpy array to df:
pca_dft=pd.DataFrame(x_trainpca,columns=["PCA1","PCA2","PCA3","PCA4"])
pca_dft["class"]=Y
pca_dft["speaker"]=sp
pca_dft.head()

/usr/local/lib/python3.10/dist-packages/sklearn/base.py:413: UserWarning: X has feature names, but PCA was fitted without feature names
  warnings.warn(


,PCA1,PCA2,PCA3,PCA4,class,speaker
0,4.348515,-1.146364,0.376333,-0.276584,1,426
1,1.560722,-0.170687,-0.249257,0.988435,1,354
2,-5.282826,0.742804,-0.319651,-0.231372,1,338
3,-0.103772,-0.603448,-0.051544,0.155027,1,376
4,0.215474,-0.824120,-0.709310,1.020471,1,367


In [32]:
#evr provides the amount of info or variance each principal component holds after projecting the data to a lower dimensional space:
print("evr is {}".format(pca.explained_variance_ratio_))

evr is [0.5290246  0.24266165 0.09908918 0.05339629]


In [33]:
param_grid= {'C':[0.1,1,10,100,1000],
             'gamma':[1,0.1,0.01,0.001,0.0001],
             'kernel':['rbf']
            }

In [34]:
from sklearn.model_selection import GridSearchCV
grid= GridSearchCV(SVC(),param_grid=param_grid,refit=True,verbose=3)

#grid.fit(Xp,Y)
grid.fit(X,Y)

#finding the best parameters:
print(grid.best_params_)

Fitting 5 folds for each of 25 candidates, totalling 125 fits
[CV 1/5] END ........C=0.1, gamma=1, kernel=rbf;, score=0.490 total time=   0.0s
[CV 2/5] END ........C=0.1, gamma=1, kernel=rbf;, score=0.490 total time=   0.0s
[CV 3/5] END ........C=0.1, gamma=1, kernel=rbf;, score=0.880 total time=   0.0s
[CV 4/5] END ........C=0.1, gamma=1, kernel=rbf;, score=0.700 total time=   0.0s
[CV 5/5] END ........C=0.1, gamma=1, kernel=rbf;, score=0.840 total time=   0.0s
[CV 1/5] END ......C=0.1, gamma=0.1, kernel=rbf;, score=0.745 total time=   0.0s
[CV 2/5] END ......C=0.1, gamma=0.1, kernel=rbf;, score=0.471 total time=   0.1s
[CV 3/5] END ......C=0.1, gamma=0.1, kernel=rbf;, score=0.600 total time=   0.0s
[CV 4/5] END ......C=0.1, gamma=0.1, kernel=rbf;, score=0.820 total time=   0.1s
[CV 5/5] END ......C=0.1, gamma=0.1, kernel=rbf;, score=0.620 total time=   0.0s
[CV 1/5] END .....C=0.1, gamma=0.01, kernel=rbf;, score=0.490 total time=   0.0s
[CV 2/5] END .....C=0.1, gamma=0.01, kernel=rbf

[CV 3/5] END .......C=1000, gamma=1, kernel=rbf;, score=0.940 total time=   0.1s
[CV 4/5] END .......C=1000, gamma=1, kernel=rbf;, score=0.980 total time=   0.0s
[CV 5/5] END .......C=1000, gamma=1, kernel=rbf;, score=0.880 total time=   0.0s
[CV 1/5] END .....C=1000, gamma=0.1, kernel=rbf;, score=0.804 total time=   0.1s
[CV 2/5] END .....C=1000, gamma=0.1, kernel=rbf;, score=0.843 total time=   0.0s
[CV 3/5] END .....C=1000, gamma=0.1, kernel=rbf;, score=0.840 total time=   0.1s
[CV 4/5] END .....C=1000, gamma=0.1, kernel=rbf;, score=0.960 total time=   0.0s
[CV 5/5] END .....C=1000, gamma=0.1, kernel=rbf;, score=0.800 total time=   0.1s
[CV 1/5] END ....C=1000, gamma=0.01, kernel=rbf;, score=0.667 total time=   0.2s
[CV 2/5] END ....C=1000, gamma=0.01, kernel=rbf;, score=0.804 total time=   0.1s
[CV 3/5] END ....C=1000, gamma=0.01, kernel=rbf;, score=0.840 total time=   0.2s
[CV 4/5] END ....C=1000, gamma=0.01, kernel=rbf;, score=0.920 total time=   0.1s
[CV 5/5] END ....C=1000, gam

In [41]:
SVM_modelpca= SVC(kernel='rbf',C=10, gamma=1, class_weight='balanced')

In [42]:
SVM_modelpca.fit(x_trainpca,Y)

SVC(C=10, class_weight='balanced', gamma=0.1)

In [43]:
Y_predict_SVMpca=SVM_modelpca.predict(x_testpca)

In [44]:
#viewing accuracy score
print("The SVM_model accuracy is given as : ",metrics.accuracy_score(Y_test,Y_predict_SVMpca))

The SVM_model accuracy is given as :  0.6964285714285714


In [45]:
print(classification_report(Y_test,Y_predict_SVMpca))

              precision    recall  f1-score   support

           0       0.70      1.00      0.82        39
           1       0.00      0.00      0.00        17

    accuracy                           0.70        56
   macro avg       0.35      0.50      0.41        56
weighted avg       0.49      0.70      0.57        56



/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
